### Working with Time Series 

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from datetime import datetime
from dateutil import parser # To parse dates from a string format

Creating a Pandas Timestamp object

In [ ]:
date = pd.to_datetime("4th of July, 2015")
date

In [ ]:
date.strftime("%A")

Alternatively...

In [ ]:
date.day_name()

We can even do NumPy-style vectorized operations on it

In [ ]:
date + pd.to_timedelta(np.arange(12), "D")

#### Pandas Time Series : Indexing by time

In [ ]:
index = pd.DatetimeIndex(["2014-07-04", "2014-08-04", "2015-07-04", "2015-08-04"])

In [ ]:
data = pd.Series([0, 1, 2, 3], index=index)
data

We can make use of any of the Series indexing pattern e.g

In [ ]:
data["2014-07-04":"2015-07-04"]

There are additional special date-only indexing operations, such as passing a year to obtain a slice of all data from that year:

In [ ]:
data["2015"]

#### Pandas Time Series Data Structures

1. Timestamps

In [ ]:
# pd.to_datetime() can parse a variety of formats a seen below
dates = pd.to_datetime(
    [datetime(2015, 7, 3), "4th of July, 2015", "2015-Jul-6", "07-07-2015", "20150708"]
)
dates

2. Time periods

In [ ]:
dates.to_period("D")

3. Time deltas

In [ ]:
dates - dates[0]

#### Regular sequences

1. Timestamps

In [ ]:
pd.date_range("2015-07-03", "2015-07-10")

2. Time periods

In [ ]:
pd.period_range("2015-07", periods=8, freq="M")

3. Time deltas

In [ ]:
pd.timedelta_range(0, periods=10, freq="H")

#### Resampling, Shifting and Windowing

In [ ]:
!pip install pandas-datareader

In [ ]:
import pandas_datareader.data as pdr
import yfinance as yf

yf.pdr_override()

goog = pdr.get_data_yahoo("GOOG", start="2004-01-01", end="2016-12-31")
goog.head()

Let's focus on the closing prices

In [ ]:
goog = goog["Close"]

Visualizing the closing price history of Google for the given timeframe

In [ ]:
goog.plot()

##### 1. Resampling

In [ ]:
goog.plot(alpha=0.5, style="-")
goog.resample("BA").mean().plot(style=":")
goog.asfreq("BA").plot(style="--")
plt.legend(["input", "resample", "asfreq"], loc="upper left")

Notice the difference at each point:\
<b>resample()</b> reports the average of the previous year\
<b>asfreq()</b> reports the value at the end of the year

asfreq() provides a method param for specifying how to impute the missing values during upsampling...see below

In [ ]:
fig, ax = plt.subplots(2, sharex=True)
data = goog.iloc[:10]
data.asfreq("D").plot(ax=ax[0], marker="o")
data.asfreq("D", method="bfill").plot(ax=ax[1], style="-o")
data.asfreq("D", method="ffill").plot(ax=ax[1], style="--o")
ax[1].legend(["back-fill", "forward-fill"])

##### 2. Shifting

Here we'll look at both shift() and tshift() by 900 days

In [ ]:
fig, ax = plt.subplots(3, sharey=True)

# applying a frequency to the data
goog = goog.asfreq("D", method="pad")

goog.plot(ax=ax[0])
goog.shift(900).plot(ax=ax[1])
goog.shift(900, freq="D").plot(ax=ax[2])

# legends and annotations
local_max = pd.to_datetime("2007-11-05")
offset = pd.Timedelta(900, "D")

ax[0].legend(["input"], loc=2)
ax[0].get_xticklabels()[4].set(weight="heavy", color="red")
ax[0].axvline(local_max, alpha=0.3, color="red")

ax[1].legend(["shift(900)"], loc=2)
ax[1].get_xticklabels()[4].set(weight="heavy", color="red")
ax[1].axvline(local_max + offset, alpha=0.3, color="red")

ax[2].legend(["tshift(900)"], loc=2)
ax[2].get_xticklabels()[1].set(weight="heavy", color="red")
ax[2].axvline(local_max + offset, alpha=0.3, color="red")

One good example of where time-shift can be used is to calculate the ROI(return on investment)\
over the course of this dataset

In [ ]:
ROI = 100 * (goog.shift(-365, freq="D") / goog - 1)
ROI.plot()
plt.ylabel("% Return on Investment")

##### 3. Rolling windows
The rolling view makes available a no. of aggregation operations by default e.g 

In [ ]:
# A one year centered rolling mean and standard deviation of the google stock prices
rolling = goog.rolling(window=365, center=True)

data = pd.DataFrame(
    {
        "input": goog,
        "one-year rolling_mean": rolling.mean(),
        "one-year rolling_std": rolling.std(),
    }
)

ax = data.plot(style=['-', '--', ':'])
ax.lines[0].set_alpha(0.3);